<a href="https://colab.research.google.com/github/essanfelix/7688581-Expert-Git-GitHub/blob/main/TP3_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice

Reprenez le dataset des diamants et créer une pipeline qui permet de trouver le meilleur modèle possible en essayant différents transformeurs et différents estimateurs.

Avez vous améliorez votre R², MedAE, MAE, MSE, RMSE ?

RMSE >> MedAE : Après pre processing, qu'en est il de cette différence ?

DIAMONDS DATASET

In [1]:
# Import des librairies

import seaborn as sns
import pandas as pd
import numpy as np


In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error

In [3]:
# Estimateurs
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [4]:
# 1/Chargement du dataset diamonds

diamonds = sns.load_dataset("diamonds")
print(diamonds.head())

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75


In [5]:
# 2/Sépararation X et y

X = diamonds.drop("price", axis=1)
y = diamonds["price"]

In [6]:
# 3/Identification colonnes numériques et catégorielles

num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object", "category"]).columns

In [7]:
# 4/Prétraitement des données : normalisation et encodage

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
])

In [9]:
# 5/Définition des modèles

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

In [11]:
# 6/Évaluation de chaque modèle avec pipeline

results = {}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

for name, model in models.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    medae = median_absolute_error(y_test, y_pred)

    results[name] = {
        "R2": r2,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MedAE": medae
    }



In [16]:
# 7/Affichage des résultats

df_results = pd.DataFrame(results).T
print("\n Résultats comparatifs :\n")
print(df_results)


 Résultats comparatifs :

                        R2         MAE           MSE         RMSE       MedAE
LinearRegression  0.918933  737.151367  1.288705e+06  1135.211645  525.998617
Ridge             0.918935  737.140156  1.288678e+06  1135.199400  525.648275
RandomForest      0.980932  270.155231  3.031257e+05   550.568483   95.870000
GradientBoosting  0.967447  398.087874  5.174831e+05   719.362950  189.748782


In [21]:
# 8/RMSE vs MedAE

print("\n Remarque :")
print(" - RMSE pénalise fortement les grosses erreurs.")
print(" - MedAE est plus résiste aux valeurs extrêmes.")
print(" Si RMSE est supérieur à MedAE, cela veut dire qu’il y a encore des prédictions avec de gros écarts.")


 Remarque :
 - RMSE pénalise fortement les grosses erreurs.
 - MedAE est plus résiste aux valeurs extrêmes.
 Si RMSE est supérieur à MedAE, cela veut dire qu’il y a encore des prédictions avec de gros écarts.
